# Randomized Truncated SVD

- subtitle: My favorite algorithm and why you should have one too.
- badges: true
- categories: [fastpages, jupyter]

In [1]:
!pip install altair

import altair as alt
import pandas as pd
import torch
import numpy as np
from scipy.sparse.linalg import svds

from collections import namedtuple

- modified from [Scikit learn's implementation](https://github.com/scikit-learn/scikit-learn/blob/15a949460dbf19e5e196b8ef48f9712b72a3b3c3/sklearn/utils/extmath.py#L246)

In [2]:
SVD = namedtuple("SVD", "U, S, V")

def randomized_range_finder(M, *, size, n_iter):
    Q = torch.randn(M.shape[1], size, device=M.device)
    for i in range(n_iter):
        Q = M.transpose(-1, -2) @ (M @ Q)
    Q, _ = torch.linalg.qr(M @ Q)
    return Q

def randomized_svd(M, n_components, *, n_oversamples=10):
    n_random = n_components + n_oversamples
    n_samples, n_features = M.shape

    Q = randomized_range_finder(M, size=n_random, n_iter=7)
    Uhat, s, Vt = torch.linalg.svd(Q.transpose(-1, -2) @ M, full_matrices=False)
    U = Q @ Uhat
    return SVD(U[:, :n_components], s[:n_components], Vt[:n_components, :])

def sample_lowrank_matrix(size, max_rank, *, noise=0.0):
    A = torch.randn(size, max_rank)
    M = A @ A.transpose(-1, -2) + noise * torch.randn(size, size) 
    return (M / size)

In [3]:
torch.manual_seed(0)
M = sample_lowrank_matrix(1000, 3)
svd_random = randomized_svd(M, 10)
svd_standard = torch.linalg.svd(M)

In [4]:
def to_numpy(svd: SVD):
    return SVD(*map(lambda t: t.cpu().numpy(), svd))

def svd_flip(svd: SVD):
    max_abs_cols = torch.argmax(np.abs(svd.U), dim=0)
    signs = torch.sign(svd.U[max_abs_cols, range(svd.U.shape[1])])
    svd.U[:] *= signs
    svd.V[:] *= signs[:, None]
    return svd

def plot_singular_values(limit_ranks, **kwargs):
    kwargs = {name: to_numpy(val) for name, val in kwargs.items()}
    data = [
        pd.DataFrame(
            {
                "Rank": np.arange(len(svd.S))[:limit_ranks], 
                "Value": svd.S[:limit_ranks], 
                "Algorithm": name,
            }
        )
    for name, svd in kwargs.items()
    ]                                                                           
    data = pd.concat(data)
        
    return (
        alt.Chart(data)
        .mark_point(size=40)
        .encode(
            x="Rank", 
            y=alt.Y("Value", axis=alt.Axis(title="Singular Value")),
            color="Algorithm", 
            shape="Algorithm",
            tooltip=["Algorithm", "Value"],
        )
    ).interactive()

def plot_singular_vector_errors(svd1, svd2, *, limit_ranks):
    svd1, svd2 = to_numpy(svd_flip(svd1)), to_numpy(svd_flip(svd2))
    errors = np.linalg.norm(svd1.U[:, :limit_ranks] - svd2.U[:, :limit_ranks], axis=0)
    data = pd.DataFrame(
        {
            "Rank": np.arange(len(errors)), "Error":errors
        }
    )
    return (
        alt.Chart(data)
        .mark_line(color="gray", strokeDash=[2,2])
        .encode(
            x="Rank", 
            y=alt.Y("Error", axis=alt.Axis(title='Error', titleColor="gray")),
            tooltip=["Error"],
        )
    ).interactive()

alt.layer(
    plot_singular_values(limit_ranks=10, **{"SVD": svd_random, "Randomized SVD": svd_standard}),
    plot_singular_vector_errors(svd_random, svd_standard, limit_ranks=10)
).resolve_scale(y="independent")

alt.LayerChart(...)

In [5]:
M = sample_lowrank_matrix(1000, 3, noise=10)
svd_random = randomized_svd(M, 10)
svd_standard = torch.linalg.svd(M)

alt.layer(
    plot_singular_values(limit_ranks=10, **{"SVD": svd_random, "Randomized SVD": svd_standard}),
    plot_singular_vector_errors(svd_random, svd_standard, limit_ranks=10)
).resolve_scale(y="independent")

alt.LayerChart(...)

In [6]:
matrix_sizes = [2**8, 2**9, 2**10, 2**11, 2**12]
n_components=2

result_cpu = []
for size in matrix_sizes:
    M = sample_lowrank_matrix(size, 3)
    time = %timeit -o randomized_svd(M, n_components=n_components)
    result_cpu.append(time.best)

result_gpu = []
for size in matrix_sizes:
    M = sample_lowrank_matrix(size, 3)
    time = %timeit -o randomized_svd(M.cuda(), n_components=n_components)
    result_gpu.append(time.best)

result_np = []
for size in matrix_sizes:
    M = sample_lowrank_matrix(size, 3)
    time = %timeit -o torch.svd(M)
    result_np.append(time.best)

result_scipy = []
for size in matrix_sizes:
    M = sample_lowrank_matrix(size, 3).numpy()
    time = %timeit -o svds(M, k=n_components)
    result_scipy.append(time.best)

df = pd.DataFrame({
    "sizes": matrix_sizes, 
    "cpu": result_cpu,
    "gpu": result_gpu,
    "full": result_np,
    "scipy": result_scipy,
    "k": n_components
})
df = df.melt(id_vars=["sizes", "k"], value_vars=["cpu", "gpu", "full", "scipy"])
df.rename(columns={"value": "runtime", "variable": "algorithm"}, inplace=True);

1000 loops, best of 5: 849 µs per loop
100 loops, best of 5: 2.76 ms per loop
100 loops, best of 5: 10.5 ms per loop
10 loops, best of 5: 43.5 ms per loop
1 loop, best of 5: 247 ms per loop
The slowest run took 554.72 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 5: 6.06 ms per loop
100 loops, best of 5: 6.46 ms per loop
100 loops, best of 5: 7.08 ms per loop
100 loops, best of 5: 10.7 ms per loop
10 loops, best of 5: 26.9 ms per loop
100 loops, best of 5: 4.91 ms per loop
10 loops, best of 5: 29.1 ms per loop
1 loop, best of 5: 275 ms per loop
1 loop, best of 5: 2.12 s per loop
1 loop, best of 5: 19.5 s per loop
1000 loops, best of 5: 1.7 ms per loop
100 loops, best of 5: 3.52 ms per loop
100 loops, best of 5: 12.1 ms per loop
10 loops, best of 5: 45.1 ms per loop
1 loop, best of 5: 253 ms per loop


In [7]:
(
    alt.Chart(df)
    .mark_point(size=40)
    .encode(
        x=alt.X("sizes", scale=alt.Scale(type='sqrt', zero=False), axis=alt.Axis(title="Matrix Size")),
        y=alt.Y("runtime", scale=alt.Scale(domain=(0, 0.3)), axis=alt.Axis(title="Runtime [s]")),
        color="algorithm", 
        shape="algorithm",
    )
).interactive()

alt.Chart(...)